In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# 0. 패키지

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_ollama import OllamaEmbeddings
import os
import re
import ollama

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_upstage import ChatUpstage

import pandas as pd

load_dotenv()

True

In [2]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
OPENAI_LLM_MODEL = 'gpt-4o-mini'
OPENAI_EMBEDDIGN_MODEL = 'text-embedding-3-large' 
UPSTAGE_EMBEDDIGN_MODEL = 'solar-embedding-1-large-passage'
# OLLAMA_EMBEDDING_MODEL = 'bge-m3'

PINECONE_INDEX_NAME_LAW = 'law'
PINECONE_INDEX_NAME_RULE = 'rule'
PINECONE_INDEX_NAME_CASE = 'case'

COHERE_API_KEY = os.getenv('COHERE_API_KEY')

In [5]:
up_embedding = UpstageEmbeddings(model=UPSTAGE_EMBEDDIGN_MODEL)
# ol_embedding = OllamaEmbeddings(model=OLLAMA_EMBEDDING_MODEL)

In [7]:
print(len(up_embedding.embed_query('주택')))
# print(len(ol_embedding.embed_query('주택')))

4096


# 0. 인덱스 초기화

In [5]:
pc = Pinecone(api_key=PINECONE_API_KEY)

index = pc.Index("law")
index.delete(delete_all=True)

{}

In [6]:
index = pc.Index("rule")
index.delete(delete_all=True)

{}

In [7]:
# index = pc.Index("case")
# index.delete(delete_all=True)

# 1. 데이터 생성

## law

In [8]:
loader = Docx2txtLoader("data/raw/law/주택임대차보호법_법률_21065_20260102.docx")
docs = loader.load()

full_text = docs[0].page_content

# "제n조" 앞에서 split (lookahead)
parts = re.split(r"\n{3,}(?=제\d+조)", full_text)

law_list = [
    Document(page_content=part.strip(), metadata={"source": "C:/Users/Admin/AI/lecNote/10_2ndProject/data/raw/law"})
    for part in parts
    if part.strip()
]

len(law_list)

43

In [9]:
law_list[42].page_content

'제31조(벌칙 적용에서 공무원 의제) 공무원이 아닌 주택임대차위원회의 위원 및 주택임대차분쟁조정위원회의 위원은 「형법」 제127조, 제129조부터 제132조까지의 규정을 적용할 때에는 공무원으로 본다.\n\n[본조신설 2016. 5. 29.]\n\n\n\n\n\n\n\n부칙 <제21065호,2025. 10. 1.>(정부조직법)\n\n제1조(시행일) 이 법은 공포한 날부터 시행한다. 다만, 부칙 제7조에 따라 개정되는 법률 중 이 법 시행 전에 공포되었으나 시행일이 도래하지 아니한 법률을 개정한 부분은 각각 해당 법률의 시행일부터 시행하고, 다음 각 호의 개정규정은 해당 호에서 정하는 날부터 시행한다.\n\n1. 다음 각 목의 개정규정은 2026년 1월 2일부터 시행한다. 다만, 부칙 제7조에 따라 개정되는 법률(가목 및 나목의 개정규정과 관련되는 부분으로 한정한다) 중 본문에 따른 시행일 전에 공포되었으나 본문에 따른 시행일이 도래하지 아니한 법률을 개정한 부분은 각각 해당 법률의 시행일부터 시행한다.\n\n가. 제19조제4항, 제23조, 제29조제1항제1호 및 제30조의 개정규정\n\n나. 제12조제2항, 제19조제3항, 제22조 및 제29조제2항 단서의 개정규정(재정경제부장관 및 재정경제부에 관한 부분으로 한정한다)\n\n다. 부칙 제7조에 따라 개정되는 법률(가목 및 나목의 개정규정과 관련되는 부분으로 한정한다)\n\n2. 생략\n\n제2조 부터 제6조까지 생략\n\n제7조(다른 법률의 개정) ①부터 <120>까지 생략\n\n<121> 주택임대차보호법 일부를 다음과 같이 개정한다.\n\n제8조의2제4항제3호 중 “기획재정부”를 “재정경제부”로 한다.\n\n<122>부터 <626>까지 생략\n\n제8조 생략\n\n\n\n법제처\n\n\n\n \n\n국가법령정보센터'

In [10]:
loader = Docx2txtLoader("data/raw/law/주택임대차보호법_시행령_35947_20260102.docx")
docs = loader.load()

full_text = docs[0].page_content

# "제n조" 앞에서 split (lookahead)
parts = re.split(r"\n{3,}(?=제\d+조)", full_text)

enforcement_decree_list = [
    Document(page_content=part.strip(), metadata={"source": "C:/Users/Admin/AI/lecNote/10_2ndProject/data/raw/law"})
    for part in parts
    if part.strip()
]

len(enforcement_decree_list)

37

In [11]:
enforcement_decree_list[36].page_content

'제35조(조정결과의 통지) ① 조정위원회는 조정절차가 종료되면 그 결과를 당사자에게 통지하여야 한다.\n\n② 조정위원회는 법 제26조제4항에 따른 조정서가 작성된 경우 조정서 정본을 지체 없이 당사자에게 교부 또는 송달하여야 한다.\n\n[본조신설 2017. 5. 29.]\n\n\n\n\n\n\n\n부칙 <제35947호,2025. 12. 30.>(재정경제부 직제)\n\n제1조(시행일) 이 영은 2026년 1월 2일부터 시행한다. 다만, 부칙 제6조에 따라 개정되는 대통령령 중 이 영 시행 전에 공포되었으나 시행일이 도래하지 않은 대통령령을 개정한 부분은 각각 해당 대통령령의 시행일부터 시행한다.\n\n제2조 부터 제5조까지 생략\n\n제6조(다른 법령의 개정) ①부터 <108>까지 생략\n\n<109> 주택임대차보호법 시행령 일부를 다음과 같이 개정한다.\n\n제17조제3항제1호 중 “기획재정부”를 “재정경제부”로 한다.\n\n<110>부터 <313>까지 생략\n\n\n\n법제처\n\n\n\n \n\n국가법령정보센터'

In [12]:
loader = Docx2txtLoader("data/raw/law/민법_임대차_발췌.docx")
docs = loader.load()

full_text = docs[0].page_content

# "제n조" 앞에서 split (lookahead)
parts = re.split(r"\n{3,}(?=제\d+조)", full_text)

civil_law_list = [
    Document(page_content=part.strip(), metadata={"source": "C:/Users/Admin/AI/lecNote/10_2ndProject/data/raw/law"})
    for part in parts
    if part.strip()
]

len(civil_law_list)

41

In [13]:
civil_law_list[33].page_content

'제648조(임차지의 부속물, 과실 등에 대한 법정질권) 토지임대인이 임대차에 관한 채권에 의하여 임차지에 부속 또는 그 사용의 편익에 공용한 임차인의 소유동산 및 그 토지의 과실을 압류한 때에는 질권과 동일한 효력이 있다.'

## rule

In [14]:
loader = Docx2txtLoader("data/raw/rule/주택임대차보호법_시행규칙.docx")
docs = loader.load()

full_text = docs[0].page_content

# "제n조" 앞에서 split (lookahead)
parts = re.split(r"\n{3,}(?=제\d+조)", full_text)

enforcement_rules_list = [
    Document(page_content=part.strip(), metadata={"source": "C:/Users/Admin/AI/lecNote/10_2ndProject/data/raw/rule"})
    for part in parts
    if part.strip()
]

len(enforcement_rules_list)

13

In [15]:
enforcement_rules_list[12].page_content

'제8조(수수료) ① 영 제7조제1항에 따른 수수료의 금액은 다음과 같다. 다만, 수수료를 계산할 때 100원 미만 금액은 계산하지 아니하고, 열람과 동시에 출력서면을 제공하는 경우에는 열람에 관한 정보제공 수수료를 별도로 계산하지 아니한다. <개정 2013. 12. 31.>\n\n1. 확정일자 부여 수수료: 1건마다 600원(계약증서가 4장을 초과할 경우 초과 4장마다 100원 추가)\n\n2. 정보제공 수수료: 1건마다 600원(출력물이 10장을 초과할 경우 초과 1장마다 50원 추가)\n\n② 제1항에도 불구하고 공증인에 대한 수수료는 「공증인 수수료 규칙」에 따른다.<신설 2013. 12. 31.>\n\n③ 영 제7조제2항에서 “「국민기초생활 보장법」에 따른 수급자 등 법무부령으로 정하는 사람”이란 다음 각 호의 어느 하나에 해당하는 사람을 말한다.<개정 2013. 12. 31., 2016. 5. 23., 2018. 10. 26.>\n\n1. 「국민기초생활 보장법」 제2조제2호에 따른 수급자\n\n2. 「독립유공자예우에 관한 법률」 제6조에 따라 등록된 독립유공자 또는 그 유족(선순위자만 해당된다)\n\n3. 「국가유공자 등 예우 및 지원에 관한 법률」 제6조에 따라 등록된 국가유공자 또는 그 유족(선순위자만 해당된다)\n\n4. 「고엽제후유의증 환자지원 등에 관한 법률」 제4조에 따라 등록된 고엽제후유증환자, 고엽제후유의증환자 또는 고엽제후유증 2세환자\n\n5. 「참전유공자예우 및 단체설립에 관한 법률」 제5조에 따라 등록된 참전유공자\n\n6. 「5ㆍ18민주유공자예우에 관한 법률」 제7조에 따라 등록 결정된 5ㆍ18민주유공자 또는 그 유족(선순위자만 해당된다)\n\n7. 「특수임무수행자 지원 및 단체설립에 관한 법률」 제6조에 따라 등록된 특수임무수행자 또는 그 유족(선순위자만 해당된다)\n\n8. 「의사상자 등 예우 및 지원에 관한 법률」 제5조에 따라 인정된 의상자 또는 의사자유족(선순위자만 해당된다)\n\n9. 「한부모가족지원법」 제5조에 따

In [16]:
loader = Docx2txtLoader("data/raw/rule/확정일자_대법원규칙_2986_20210610.docx")
docs = loader.load()

full_text = docs[0].page_content

# "제n조" 앞에서 split (lookahead)
parts = re.split(r"\n{3,}(?=제\d+조)", full_text)

confirmation_list = [
    Document(page_content=part.strip(), metadata={"source": "C:/Users/Admin/AI/lecNote/10_2ndProject/data/raw/rule"})
    for part in parts
    if part.strip()
]

len(confirmation_list)

15

In [17]:
confirmation_list[14].page_content

'제14조(대법원예규 위임) 확정일자 부여 등의 절차와 관련하여 필요한 사항 중 이 규칙에서 정하고 있지 아니한 사항은 대법원예규로 정할 수 있다.\n\n\n\n\n\n\n\n부칙 <제2986호,2021. 5. 27.>(부동산등기규칙)\n\n제1조(시행일) 이 규칙은 2021년 6월 10일부터 시행한다.\n\n제2조 생략\n\n제3조(다른 규칙의 개정) ①부터 ③까지 생략\n\n④ 주택임대차계약증서의 확정일자 부여 및 정보제공에 관한 규칙 일부를 다음과 같이 개정한다.\n\n제6조제4항제1호 중 “것을”을 “것으로서 법원행정처장이 지정ㆍ공고하는 인증서를”로 하고, 같은 조에 제5항을 신설한다.\n\n⑤ 제4항제1호의 공고는 인터넷등기소에 하여야 한다.\n\n\n\n법제처\n\n\n\n \n\n국가법령정보센터'

## case

In [18]:
loader = Docx2txtLoader("data/raw/case/전세피해법률상담사례집.docx")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,   
                                               chunk_overlap=200,
                                               separators=['\n\n\n\n\n\n','\n\n', '\n', ' ', '']) 
legal_counsel_list = loader.load_and_split(text_splitter=text_splitter)
len(legal_counsel_list)

52

In [19]:
legal_counsel_list[49].page_content

'이러한 경우를 방지하기 위하여 소유자가 가등기를 하지 못하도록 법적으로 막아야 하는데, "가등기 설정금지"특약을 하는 방법밖에 없고 이를 위반해도 지정할 방법이 따로 없으니, 전세권등기를 설정해 놓으면 임차 후 가등기를 하여도 전세권자의 배당요구에 의하여 전세권이 소멸하는 결과 이보다 후순위인 소유권이전가등기는 말소되는 것이기 때문에 전세시 물권인 전세권등기를 하는 것이 필요한 이유입니다.\n\n소유권이전가등기란 본등기인 소유권이전등기를 하는 데 필요한 형식적 요건이나 실질적 요건이 구비되지 않았을 때, 장차의 본등기의 순위보전을 위하여 미리 해두는 등기를 말하며, 가등기에는 담보가등기(저당권과 유사)와 소유권이전청구권가등기의 두 가지가 있습니다.'

In [20]:
loader = Docx2txtLoader("data/raw/case/전세피해지원사례집_2025.docx")
docs = loader.load()

full_text = docs[0].page_content

parts = re.split(r"\n{6,}(?=\d+\.)", full_text)

legal_support_list = [
    Document(page_content=part.strip(), metadata={"source": "C:/Users/Admin/AI/lecNote/10_2ndProject/data/raw/case"})
    for part in parts
    if part.strip()
]
len(legal_support_list)

22

In [21]:
legal_support_list[19].page_content

'20. 임대인의 회생 또는 파산 문제\n\n임대인이 회생(파산)절차에 들어갔는데, 보증금도 회생(파산)절차에 따라 면책되는 건가요?\n\n확정일자를 부여받은 임차인이 주택 점유 및 전입신고를 마친 경우, 대항력과 우선변제권을 갖습니다.\n보증금이 확정일자를 갖춘 경우, 파산재산에 속하는 주택의 환가대금에서 후순위 권리자보다 우선 변제를 받을 수 있습니다.\n\n다만, 말소기준권리보다 대항력이 후순위인 임차인의 경우, 경매 배당순위에 따라 보증금 손실이 발생할 수 있으며,\n회생 또는 파산절차 진행에 따라 보증금 손실이 발생할 수 있습니다.\n\n회생·파산절차 관련 법률상담이 필요한 경우 HUG 전세피해지원센터 또는 지자체 전세피해지원센터를 통해 무료 상담을 받을 수 있습니다.'

# 2. 함수

In [14]:
import re

def remove_special_chars(text:str) -> str:
    '특수문자 및 \n제거(:는 그대로)'
    # \n 제거
    text = text.replace('\n',' ')
    
    # 한글, 영문, 숫자, 공백, 마침표, 콤마, :만 남기고 전부 제거
    cleaned = re.sub(r'[^가-힣a-zA-Z0-9\s,.:]', '', text) 
    
    # 불필요한 중복 공백을 제거
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()
    return cleaned

In [15]:
# 제목을 추출하는 함수(exaone3.5) : cmd창에서 ollama pull exaone3.5:2.4b

from langchain_ollama import ChatOllama

def extract_title_with_llm(content):
    'exaone3.5를 사용하여 content의 제목을 추출하는 함수'
    title_extractor_llm = ChatOllama(model='exaone3.5:2.4b',
                                     temperature=0.1,
                                     num_predict=30, # 최대 30토큰까지만 출력
                                     )
    # content = remove_special_chars(content)
    prompt = f'''다음 소득세법 조문의 핵심 제목을 30토큰 이내로 간단히 완벽하게 말이 되도록 추출하세요. 중간에 말이 끊기면 안됩니다
    조문 : {content}'''
    ai_message = title_extractor_llm.invoke(prompt)
    title = ai_message.content.strip()
    return remove_special_chars(title)

In [17]:
def get_law_category():
    """
    주택임대차 RAG 시스템을 위한 카테고리-키워드 가중치 사전
    Returns:
        dict: {카테고리명: {키워드: 가중치}}
    """
    return {
        # 1. 보증금 보호 (가장 치명적인 리스크)
        '보증금_대항력': {
            '보증금': 3, '전세금': 2, '보증금반환': 3, '반환청구': 2,
            '대항력': 3, '전입신고': 2, '주민등록': 2, '점유': 2, '인도': 2,
            '우선변제권': 3, '최우선변제': 3, '소액임차인': 3, '배당': 2, '경매': 2, '공매': 2,
            '확정일자': 3, '확정일자부': 2,'임차권등기': 3, '임차권등기명령': 3,
            '반환보증': 2, '보증보험': 2, 'HUG': 1, '보증보험': 2,
            '대항요건': 3,'제삼자': 2,'효력': 1,'효력발생': 2, '입주': 1,
            '거주': 1,'주택인도': 2, '임대차계약증서': 2,'환가대금': 2,
            '후순위권리자': 2,'변제': 1,'배당요구': 2,'임차권등기명령': 3,'임대차등기': 2,
            '지방법원': 2,'등기촉탁': 1,'등기신청': 1,
        },

        # 2. 계약 갱신 및 거절 (가장 빈번한 분쟁)
        '계약갱신': {
            '계약갱신': 3, '갱신요구': 3, '갱신요구권': 3, '갱신거절': 3, '묵시적갱신': 3,
            '계약연장': 2, '재계약': 2, '존속기간': 2, '2년': 1,
            '실거주': 3, '거절사유': 2, '손해배상': 2,
            '연체': 2, '2기': 2, '의무위반': 2,
            '재계약': 2, '갱신청구': 2, '거절사유': 2, '복비': 1, '계약갱신청구권': 3,
            '갱신거절통지': 3,'계약조건변경': 2, '계약만료': 2,'직계존속': 2,
            '직계비속': 2,'직계존비속': 2, '중개보수': 1,
        },

        # 3. 계약 해지 및 종료 (탈출 전략)
        '계약해지': {
            '계약해지': 3, '해지통고': 3, '중도해지': 3, '기간만료': 2,
            '계약종료': 2, '퇴거': 2, '이사': 1, '명도': 2,
            '합의해지': 2, '통지': 1, '3개월': 2, '즉시해지': 3
        },

        # 4. 차임(월세) 및 보증금 증감
        '임대료_증감': {
            '월세전환': 3, '반전세': 2, '차임': 3, '월세': 2, '증액': 3, 
            '5퍼센트': 3, '5%': 3, '20분의 1': 2, '상한': 2, '감액': 2, '인상': 2, '1년': 2, '1년 이내': 2,
            '전환율': 2, '월차임': 2, '경제사정': 1, '부담': 1, '전환되는 금액': 2,
            '초과차임': 2, '반환': 2, '증액청구': 3,'증액비율': 2, '차임액': 2,
            '보증금증액': 2, '차임연체': 2, '2기차임': 2,'연체': 1,
        },

        # 5. 유지보수 및 원상회복 (민법 영역)
        '수선_원상회복': {
            '수선': 3, '수리': 3, '원상회복': 3, '파손': 2, '훼손': 2, '유지': 2, '하자': 2,
            '누수': 2, '곰팡이': 2, '보일러': 2, '필요비': 3, '유익비': 3,
            '비용상환': 2, '보존행위': 2, '관리비': 1
        },

        # 6. 전세사기 및 권리 리스크 (독소조항 탐지용)
        '권리_리스크': {
            '전세사기': 3, '깡통전세': 3, '신탁': 3, '근저당': 3, '저당권': 3, '신탁부동산': 3,
            '선순위': 3, '가압류': 3, '압류': 3, '국세': 2, '지방세': 2,
            '체납': 3, '납세증명': 2, '위반건축물': 2, '불법건축물': 2,
            '특약': 2, '독소조항': 2, '강행규정': 3, '효력이 없다': 3, 
            '무효': 3, '불리한 약정': 3, '편면적 강행규정': 2, '등기부등본': 2,
            '말소기준권리': 3, '채권최고액': 2, '가등기': 2, '전세권': 2, '전세권등기': 2,
            '소유권이전등기': 2,'말소': 2, '배당요구': 2,'건축물대장': 2, '보존등기': 2, '이행강제금': 1,
        },

        # 7. 행정 절차 (Rules Index 연결)
        '행정절차': {
            '확정일자부여': 3, '동주민센터': 2, '등기소': 2, '인터넷등기소': 3,
            '수수료': 1, '열람': 2, '제공요청': 2, '이해관계인': 2,
            '임대차계약증서': 2, '전자계약': 2, '신분증': 1, '등기부등본': 3,
            '건축물대장': 3,'중개대상물확인설명서': 2,'공인중개사': 1,'중개수수료': 2, '중개보수': 2,
        },

        # 8. 소송 및 분쟁 해결 (최후 수단)
        '분쟁해결': {
            '분쟁조정': 3, '조정위원회': 2, '지급명령': 3, '소송': 3, '조정': 3, '조정부': 2,
            '판결': 2, '집행권원': 2, '경매': 3, '공매': 2,
            '내용증명': 2, '손해배상': 3, '지연이자': 2, '조정신청': 3, '신청': 2, '각하': 2,
            '조정절차': 3, '송달': 2, '서면': 2, '구두': 2,
            '처리기간': 2, '60일': 2, '30일': 1,
            '심사관': 1, '조사관': 1, '사무국': 1,
            '조정신청': 3,'조정안': 2,'조정서': 2, '조정성립': 2,'인도명령': 2,'명도소송': 2,
            
            # 소송/집행 트리거
            '지급명령': 3, '소송': 3, '판결': 2, '집행권원': 2,
            '경매': 3, '공매': 2, '내용증명': 2, '손해배상': 3, '지연이자': 2,

            # 시행령(원문 표기): 조정 대상 범위 + 신청 절차 상세
            '제22조(조정위원회의 심의ㆍ조정 사항)': 3,
            '제29조(조정부의 구성 및 운영)': 1,
            '제30조(조정의 신청)': 2,
            '제31조(조정신청인에게 안내하여야 할 사항)': 1,
            '제32조(조정서류의 송달 등)': 1,
        },
        
        # 9. 임차권 승계 및 가족 
        '임차권_승계': {
            '임차권승계': 3, '승계': 3, '사망': 3, '상속': 3, '상속인': 3,
            '사실혼': 3, '배우자': 2, '가정공동생활': 3, '2촌': 2, 
            '공동상속': 2, '반환청구권': 2
        },
        
        # 10. 전대/양도
        '전대_양도': {
            '전대': 3, '전차': 2, '전대차': 2, '임대인의 동의': 3,
            '양도': 2, '임차권 양도': 2
        },

        # 0. 적용 범위/예외(필요 시 사용)
        '적용범위_예외': {
            '적용범위': 3, '주거용': 2, '주택 일부': 2, '주거 외 목적': 2,
            '일시사용': 3, '일시사용하기 위한 임대차': 3,
            '미등기': 3, '미등기 전세': 3, '전세계약': 2
        },
    }

In [18]:
def categorize_content(content, top_k=None):
    '''
    내용 카테고리 분류 - 점수 기반으로 모든 카테고리를 점수 순으로 반환
    Parameters : 
        - content : 분류할 텍스트 내용
        - top_k : 상위 몇개까지 카테고리를 반환할지(None이면 모든 카테고리 반환)
        
    Returns :
        - 카테고리 리스트(점수 높은 순)
    '''
    category_keywords = get_law_category()
    category_scores = {}
    
    # 각 카테고리별 점수 계산
    for category, weighted_keywords in category_keywords.items():
        # print(category, weighted_keywords)
        score = 0
        for keyword, weight in weighted_keywords.items():
            # if keyword in content: # content에 keyword가 포함되어있는지 여부
                # score += weight
            count = content.count(keyword) # content에 keyword가 몇번 나오는지
            score += count * weight
        if score > 0 :
            category_scores[category] = score
    # print(category_scores)
    
    # 내림차순 정렬한 카테고리 이름만 추출
    sorted_categories = sorted(category_scores.items(), key=lambda x : x[1], reverse=True)
    all_categories = [category[0] for category in sorted_categories]
    # print(all_categories)
    
    # 매칭되는 카테고리가 없으면 '기타' 반환
    if not all_categories:
        all_categories = ['기타']
        
    # top_k가 지정되면 상위 top_k개만, 아니면 전체 반환
    if top_k is not None:
        return all_categories[:top_k]
    return all_categories

In [20]:
# 해당 조항 추출하기

def get_article(content):
    '조항들 추출'
    article_match = re.findall(r'제(\d+)조', content)
    article_match =list(set(article_match)) # 중복 제거
    article_match.sort(key=lambda x : int(x))
    if article_match:
        return '「'+ ', '.join([f'{a}조' for a in article_match]) +'」'

In [21]:
# 참조조항을 추출하는 함수

def extract_articles_from_docs(documents):
    '''검색된 문서들(documents)에서 aritcle 정보를 추출하여 리스트로 변환
    Returns : 중복이 제거된 조항 리스트 ex : ['제4조', '제16조']'''
    articles = []
    for doc in documents:
        article = doc.metadata.get('article', '조항없음')
        
        # 「」제거. 문자를 list로 분리
        if article != '조항없음':
            article = article.replace('「','').replace('」','')
            article_list = article.split(', ')
            articles.extend(article_list)
            
    # 중복 제거 및 정렬
    unique_articles = list(set(articles))
    unique_articles.sort(key=lambda x : int(x[:-1]))
    final_articles = ['제'+article for article in unique_articles] 
    return '주택임대차보호법' + ','.join(final_articles)

In [22]:
# 검색된 document를 텍스트로 변환하는 함수
def format_documents(documents):
    return '\n\n---\n\n'.join([doc.page_content for doc in documents])

# 3. metadata 추가하기

## law

In [23]:
# 주택임대차보호법_법률
law_enhanced_chunks = []
for i, chunk in enumerate(law_list):
    content = chunk.page_content
    metadata = chunk.metadata.copy()
    
    # metadata['title'] = extract_title_with_llm(content) # exaone으로 제목 추출
    metadata['category'] = categorize_content(content) # categroy 추출
    metadata['chunk_id'] = f'chunk_{i:02d}'
    metadata['priority'] = 1
    metadata['src_title'] = '주택임대차보호법_법률(제21065호)(20260102)'
    article_match = get_article(content)
    if article_match:
        metadata['article'] = article_match
    law_enhanced_chunks.append(type(chunk)(page_content=content, metadata=metadata)) # type(chunk)는 Document타입
print('확장된 chunk 처리 완료')
print(len(law_enhanced_chunks))

확장된 chunk 처리 완료
43


In [24]:
# 주택임대차보호법_시행령
enforcement_decree_enhanced_chunks = []
for i, chunk in enumerate(enforcement_decree_list):
    content = chunk.page_content
    metadata = chunk.metadata.copy()
    
    # metadata['title'] = extract_title_with_llm(content) # exaone으로 제목 추출
    metadata['category'] = categorize_content(content) # categroy 추출
    metadata['chunk_id'] = f'chunk_{i:02d}'
    metadata['priority'] = 2
    metadata['src_title'] = '주택임대차보호법_시행령(대통령령)(제35947호)(20260102)'
    article_match = get_article(content)
    if article_match:
        metadata['article'] = article_match
    enforcement_decree_enhanced_chunks.append(type(chunk)(page_content=content, metadata=metadata)) # type(chunk)는 Document타입
print('확장된 chunk 처리 완료')
print(len(enforcement_decree_enhanced_chunks))

확장된 chunk 처리 완료
37


In [25]:
# 민법 임대차
civil_enhanced_chunks = []
for i, chunk in enumerate(civil_law_list):
    content = chunk.page_content
    metadata = chunk.metadata.copy()
    
    # metadata['title'] = extract_title_with_llm(content) # exaone으로 제목 추출
    metadata['category'] = categorize_content(content) # categroy 추출
    metadata['chunk_id'] = f'chunk_{i:02d}'
    metadata['priority'] = 4
    metadata['src_title'] = '민법_임대차(제20432호)(20260101)'
    article_match = get_article(content)
    if article_match:
        metadata['article'] = article_match
    civil_enhanced_chunks.append(type(chunk)(page_content=content, metadata=metadata)) # type(chunk)는 Document타입
print('확장된 chunk 처리 완료')
print(len(civil_enhanced_chunks))

확장된 chunk 처리 완료
41


## rule

In [31]:
# 주택임대차보호법_시행규칙
enforcement_rules_enhanced_chunks = []
for i, chunk in enumerate(enforcement_rules_list):
    content = chunk.page_content
    metadata = chunk.metadata.copy()
    
    # metadata['title'] = extract_title_with_llm(content) # exaone으로 제목 추출
    metadata['category'] = categorize_content(content) # categroy 추출
    metadata['chunk_id'] = f'chunk_{i:02d}'
    metadata['priority'] = 3
    metadata['src_title'] = '주택임대차보호법_시행규칙(제1022호)(20220207)'
    article_match = get_article(content)
    if article_match:
        metadata['article'] = article_match
    enforcement_rules_enhanced_chunks.append(type(chunk)(page_content=content, metadata=metadata)) # type(chunk)는 Document타입
print('확장된 chunk 처리 완료')
print(len(enforcement_rules_enhanced_chunks))

확장된 chunk 처리 완료
13


In [32]:
# 확정일자 대법원 규칙
confirmation_enhanced_chunks = []
for i, chunk in enumerate(confirmation_list):
    content = chunk.page_content
    metadata = chunk.metadata.copy()
    
    # metadata['title'] = extract_title_with_llm(content) # exaone으로 제목 추출
    metadata['category'] = categorize_content(content) # categroy 추출
    metadata['chunk_id'] = f'chunk_{i:02d}'
    metadata['priority'] = 3
    metadata['src_title'] = '확정일자_대법원규칙(제2986호)(20210610)'
    article_match = get_article(content)
    if article_match:
        metadata['article'] = article_match
    confirmation_enhanced_chunks.append(type(chunk)(page_content=content, metadata=metadata)) # type(chunk)는 Document타입
print('확장된 chunk 처리 완료')
print(len(confirmation_enhanced_chunks))

확장된 chunk 처리 완료
15


## case

In [34]:
# 전세피해법률상담사례
legal_counsel_enhanced_chunks = []
for i, chunk in enumerate(legal_counsel_list):
    if i%7 == 0:
        print(f" 진행 중 : {i/len(legal_counsel_list)*100:.2f} % 진행")
    content = chunk.page_content
    metadata = chunk.metadata.copy()
    
    metadata['src_title'] = extract_title_with_llm(content) # exaone으로 제목 추출
    metadata['category'] = categorize_content(content) # categroy 추출
    metadata['chunk_id'] = f'chunk_{i:02d}'
    metadata['priority'] = 7
    metadata['src_title'] = extract_title_with_llm(content)
    article_match = get_article(content)
    if article_match:
        metadata['related_article'] = article_match
    legal_counsel_enhanced_chunks.append(type(chunk)(page_content=content, metadata=metadata)) 
print('확장된 chunk 처리 완료')
print(len(legal_counsel_enhanced_chunks))

 진행 중 : 0.00 % 진행
 진행 중 : 38.46 % 진행
 진행 중 : 76.92 % 진행
확장된 chunk 처리 완료
52


In [35]:
%%time
# 전세피해지원사례
legal_support_enhanced_chunks = []
for i, chunk in enumerate(legal_support_list):
    if i%3 == 0:
        print(f" 진행 중 : {i/len(legal_support_list)*100:.2f} % 진행")
    content = chunk.page_content
    metadata = chunk.metadata.copy()
    
    metadata['src_title'] = extract_title_with_llm(content) # exaone으로 제목 추출
    metadata['category'] = categorize_content(content) # categroy 추출
    metadata['chunk_id'] = f'chunk_{i:02d}'
    metadata['priority'] = 7
    metadata['src_title'] = extract_title_with_llm(content)
    article_match = get_article(content)
    if article_match:
        metadata['related_article'] = article_match
    legal_support_enhanced_chunks.append(type(chunk)(page_content=content, metadata=metadata)) 
print('확장된 chunk 처리 완료')
print(len(legal_support_enhanced_chunks))

 진행 중 : 0.00 % 진행
 진행 중 : 90.91 % 진행
확장된 chunk 처리 완료
22


## pinecone 업로드

In [26]:
%%time

pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))

all_docs = (
    law_enhanced_chunks
    + enforcement_decree_enhanced_chunks
    + civil_enhanced_chunks
)

# 데이터를 처음 업로드할 때
law_2_database = PineconeVectorStore.from_documents(documents=all_docs,
                                              embedding=up_embedding, 
                                              # embedding=ol_embedding,
                                              index_name='law-2')

CPU times: total: 3.97 s
Wall time: 25.4 s


In [33]:
%%time

pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))

all_docs = (
    enforcement_rules_enhanced_chunks
    + confirmation_enhanced_chunks
)

# 데이터를 처음 업로드할 때
index_name = 'rule'
rule_database = PineconeVectorStore.from_documents(documents=all_docs,
                                              embedding=up_embedding, 
                                              # embedding=ol_embedding,
                                              index_name=PINECONE_INDEX_NAME_RULE)

CPU times: total: 875 ms
Wall time: 8.23 s


In [36]:
%%time

pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))

all_docs = (
    legal_counsel_enhanced_chunks
    + legal_support_enhanced_chunks
)

# 데이터를 처음 업로드할 때
index_name = 'case'
case_database = PineconeVectorStore.from_documents(documents=all_docs,
                                              embedding=up_embedding, 
                                              # embedding=ol_embedding,
                                              index_name=PINECONE_INDEX_NAME_CASE)

CPU times: total: 2.3 s
Wall time: 19.9 s


In [45]:
law_database

## pinecone 불러오기

In [12]:
# 업로드한 벡터DB를 가져올 때

law_database = PineconeVectorStore(embedding=up_embedding, # 질문을 임베딩하여 유사도 검색
                               # embedding=ol_embedding,
                               index_name=PINECONE_INDEX_NAME_LAW)

rule_database = PineconeVectorStore(embedding=up_embedding, # 질문을 임베딩하여 유사도 검색
                                    # embedding=ol_embedding,
                                    index_name=PINECONE_INDEX_NAME_RULE)

case_database = PineconeVectorStore(embedding=up_embedding, # 질문을 임베딩하여 유사도 검색
                                    # embedding=ol_embedding,
                                    index_name=PINECONE_INDEX_NAME_CASE)

# 4. csv

In [39]:
def documents_to_df(docs):
    rows = []
    for d in docs:
        row = {}

        # metadata 풀기
        if hasattr(d, "metadata") and isinstance(d.metadata, dict):
            row.update(d.metadata)

        # page_content 넣기
        row["page_content"] = getattr(d, "page_content", "")

        rows.append(row)

    return pd.DataFrame(rows)

In [83]:
# law
# docs = [Document(...), Document(...), ...]

law_df = documents_to_df(law_enhanced_chunks)   
enforcement_decree_df = documents_to_df(enforcement_decree_enhanced_chunks)
civil_law_df = documents_to_df(civil_enhanced_chunks)

# display(law_df.head(1))
# display(enforcement_decree_df.head(1))
# display(civil_law_df.head(1))

law = pd.concat([law_df, enforcement_decree_df, civil_law_df]).reset_index(drop=True)
law.drop('page_content', inplace=True, axis=1)
law = law[['article', 'category','chunk_id','priority', 'source', 'src_title', 'text']]
law.head(1)

,article,category,chunk_id,priority,source,src_title,text
0,NaN,[주택유형],chunk_00,1,C:/Users/Admin/AI/lecNote/10_2ndProject/data/r...,주택임대차보호법_법률(제21065호)(20260102),주택임대차보호법\n\n주택임대차보호법 ( 약칭: 주택임대차법 )\n\n[시행 202...


In [85]:
# rule

enforcement_rules_df = documents_to_df(enforcement_rules_enhanced_chunks)
confirmation_df = documents_to_df(confirmation_enhanced_chunks)

# display(enforcement_rules_df.head(1))
# display(confirmation_df.head(1))

rule = pd.concat([enforcement_rules_df, confirmation_df]).reset_index(drop=True)
rule.drop('page_content', inplace=True, axis=1)
rule = rule[['article', 'category','chunk_id','priority', 'source', 'src_title', 'text']]
rule.head(1)

,article,category,chunk_id,priority,source,src_title,text
0,NaN,"[우선변제권, 주택유형]",chunk_00,3,C:/Users/Admin/AI/lecNote/10_2ndProject/data/r...,주택임대차보호법_시행규칙(제1022호)(20220207),주택임대차계약증서상의 확정일자 부여 및 임대차 정보제공에 관한 규칙\n\n주택임대차...


In [93]:
# case

legal_counsel_df = documents_to_df(legal_counsel_enhanced_chunks)
legal_support_df = documents_to_df(legal_support_enhanced_chunks)

# display(legal_counsel_df.head(1))
# display(legal_support_df.head(1))

case = pd.concat([legal_counsel_df, legal_support_df]).reset_index(drop=True)
case.drop('page_content', inplace=True, axis=1)
case = case[['category','chunk_id','priority', 'related_article', 'source', 'src_title', 'text']]
case.head(1)

,category,chunk_id,priority,related_article,source,src_title,text
0,"[차임보증금, 담보권리, 신탁대리, 주택유형, 독소조항, 우선변제권, 대항력, 소액...",chunk_00,7,NaN,data/raw/case/전세피해법률상담사례집.docx,전세사기 예방 핵심 유형: 1. 깡통전세: 보증금이 주택가격보다 낮은 경우 주의.,1. 전세(임대차)사기 유형 종합분석\n\n\n\n전세사기 피해자들의 상담사례를 분...


## 저장

In [94]:
law.to_csv('data/processed/csv/law.csv', index=False, encoding='utf-8')
rule.to_csv('data/processed/csv/rule.csv', index=False, encoding='utf-8')
case.to_csv('data/processed/csv/case.csv', index=False, encoding='utf-8')

## 불러오기

In [95]:
law = pd.read_csv('data/processed/csv/law.csv')
rule = pd.read_csv('data/processed/csv/rule.csv')
case = pd.read_csv('data/processed/csv/case.csv')

In [90]:
case.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   source           74 non-null     object
 1   src_title        74 non-null     object
 2   category         74 non-null     object
 3   chunk_id         74 non-null     object
 4   priority         74 non-null     int64 
 5   text             74 non-null     object
 6   related_article  9 non-null      object
dtypes: int64(1), object(6)
memory usage: 4.2+ KB


# 5. 확인용

In [97]:
query = '벽지 등이 손상될 경우 세입자가 전부 수리하고 나간다'

In [99]:
retriever = law_database.as_retriever(search_kwargs={'k':3, 'filter':{'category':{'$in':categorize_content(query)}}})
docs = retriever.invoke(query)
print('관련문서')
for i, doc in enumerate(docs):
    category = doc.metadata['category']
    article  = doc.metadata.get('article', '조항없음')
    chunk_id = doc.metadata.get('chunk_id')
    text    = doc.metadata.get('text')
    print(f'{i+1}.{chunk_id}-{article} {category} - {text}')

관련문서
1.chunk_19-「634조」 ['기타'] - None
2.chunk_20-「10조」 ['기타'] - None
3.chunk_02-「615조」 ['기타'] - None


In [103]:
# 1. LLM과 임베딩 초기화
llm = ChatOpenAI(model=OPENAI_LLM_MODEL)
up_embedding = UpstageEmbeddings(model=UPSTAGE_EMBEDDIGN_MODEL)
ol_embedding = OllamaEmbeddings(model=OLLAMA_EMBEDDING_MODEL)

# 2. 업로드한 벡터 DB에서 가져올 때
law_database= PineconeVectorStore(
    embedding=up_embedding,
    # embedding=ol_embedding,
    index_name='law')

rule_database = PineconeVectorStore(
    embedding=up_embedding,
    # embedding=ol_embedding,
    index_name='rule')

case_database = PineconeVectorStore(
    embedding=up_embedding,
    # embedding=ol_embedding,
    index_name='case')

In [107]:
# 키워드사전
keyword_dict = [
    # 사람
    "세입자 -> 임차인",          # 임차인(賃借人) :contentReference[oaicite:4]{index=4}
    "세들어사는사람 -> 임차인",
    "임차자 -> 임차인",
    "입주자 -> 임차인",          # 문서에도 ‘입주자’ 표현이 나오긴 하지만 맥락상 임차인 질의로 정규화용

    "집주인 -> 임대인",          # 임대인(賃貸人) :contentReference[oaicite:5]{index=5}
    "임대인 -> 임대인",
    "주인 -> 임대인",

    # 금액/지급
    "전세금 -> 임대차의 보증금",  # “전세금”은 “임대차의 보증금”으로 본다 :contentReference[oaicite:6]{index=6}
    "보증금 -> 보증금",          # 문서 그대로 :contentReference[oaicite:7]{index=7}
    "월세 -> 차임",              # 차임(借賃) :contentReference[oaicite:8]{index=8}
    "월임대료 -> 차임",
    "임대료 -> 차임",
    "렌트비 -> 차임",

    "보증금돌려받기 -> 보증금 반환",
    "보증금반환 -> 보증금 반환",
    "보증금못받음 -> 보증금 반환",

    # 권리/요건
    "전입신고 -> 주민등록",      # 전입신고를 한 때에 주민등록이 된 것으로 본다 :contentReference[oaicite:9]{index=9}
    "전입 -> 주민등록",
    "확정일자 -> 확정일자",      # 확정일자 부여/정보제공 :contentReference[oaicite:10]{index=10}
    "대항요건 -> 대항요건",
    "대항력 -> 대항력",          # 제3조(대항력 등) :contentReference[oaicite:11]{index=11}
    "우선변제 -> 우선변제",
    "우선변제권 -> 우선변제권",

    # 절차/사건
    "조정위 -> 주택임대차분쟁조정위원회",  # :contentReference[oaicite:14]{index=14}
    "분쟁조정 -> 주택임대차분쟁조정위원회",
]

prompt = ChatPromptTemplate.from_template(f'''사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요. 
만약 변경할 필요가 없을 경우 사용자의 질문을 변경하지 마세요. 그런 경우에는 질문만 리턴해주세요
사전 : {keyword_dict}
질문 : {{question}}''')

keyword_chain = prompt | llm | StrOutputParser()
keyword_chain.invoke({'question':'벽지 등이 손상될 경우 세입자가 전부 수리하고 나간다'})

'벽지 등이 손상될 경우 임차인이 전부 수리하고 나간다'

In [108]:
# 4. Retriever 생성 (category중 하나라도 포함되어있는 chunk만)
retriever = law_database.as_retriever(search_kwargs = {'k':3, 'filter':{'category':{'$in':categorize_content(query)}}})

# 5. 프롬프트 템플릿
template = f'당신은 최고의 한국 주택임대차보호법 전문가입니다. 다음 문맥을 참고하여 질문에 답하세요. 답을 모르면 모른다고 답하세요. 최대 3문장으로 간결하게 답변하세요. 질문 : {{query}} / 문맥 : {{context}} / 답변 :'
prompt = ChatPromptTemplate.from_template(template)

# 6. 검색된 document를 텍스트로 변환하는 함수
def format_documents(documents):
    return '\n\n---\n\n'.join([doc.page_content for doc in documents])

In [109]:
# 7. RAG 체인 구성 (LCEL 방식)
from langchain_core.runnables import RunnablePassthrough # {'query':'~'} → '~'

rag_chain = ({'context':retriever | format_documents, 'query':RunnablePassthrough()} # 질문 그대로 전달
             | prompt # prompt에 context와 query 변수 주입
             | llm    # llm에 prompt invoke
             | StrOutputParser()) 

final_chain = keyword_chain | rag_chain

# 8. 실행
query = '벽지 등이 손상될 경우 세입자가 전부 수리하고 나간다'
result = final_chain.invoke(query)
print(result + '\n\n*위의 답변은 AI에 의해 생성된 답변이므로 약간의 오차가 발생할 수 있습니다.*')

임차인은 벽지 등의 손상이 발생한 경우 이를 수리할 의무가 있지만, 임대인에게 이러한 손상을 통지해야 합니다. 만약 임대인이 이미 손상을 알고 있다면, 임차인은 통지의 의무가 면제됩니다. 그러나 임차인이 수리를 전부 수행하고 나갈 경우, 임대인에게 부속물 매수청구권이 발생하지는 않습니다.

*위의 답변은 AI에 의해 생성된 답변이므로 약간의 오차가 발생할 수 있습니다.*


In [25]:
# 재사용 가능한 함수로 만들기

def ask_with_reference(query:str, k:int=4):
    'query, 표준화된 query, 답변, 참조사항 출력'
    
    # 1. llm, embedding
    llm = ChatOpenAI(model=OPENAI_LLM_MODEL)
    up_embedding = UpstageEmbeddings(model=UPSTAGE_EMBEDDIGN_MODEL)
    ol_embedding = OllamaEmbeddings(model=OLLAMA_EMBEDDING_MODEL)

    # 2. 업로드한 벡터 DB에서 가져올 때
    law_database= PineconeVectorStore(
        embedding=up_embedding,
        # embedding=ol_embedding,
        index_name=PINECONE_INDEX_NAME_LAW)

    rule_database = PineconeVectorStore(
        embedding=up_embedding,
        # embedding=ol_embedding,
        index_name=PINECONE_INDEX_NAME_RULE)

    case_database = PineconeVectorStore(
        embedding=up_embedding,
        # embedding=ol_embedding,
        index_name=PINECONE_INDEX_NAME_CASE)
    
    # 3. keyword_chain 
    keyword_dict = [
        # 사람
        "세입자 -> 임차인",
        "월세입자 -> 임차인",
        "세들어사는사람 -> 임차인",
        "임차자 -> 임차인",
        "입주자 -> 임차인",          

        "집주인 -> 임대인",          
        "임대인 -> 임대인",
        "주인 -> 임대인",
        "건물주 -> 임대인", 
        "임대업자 -> 임대인",
        
        "부동산 -> 공인중개사", 
        "중개인 -> 공인중개사",

        # 금액/지급
        "전세금 -> 임차보증금",  
        "보증금 -> 임차보증금",  
        "월세 -> 차임",    
        "월임대료 -> 차임",
        "임대료 -> 차임",
        "렌트비 -> 차임",
        "방세 -> 차임",
        "관리비 -> 관리비용",

        "보증금돌려받기 -> 보증금 반환",
        "보증금반환 -> 보증금 반환",
        "보증금못받음 -> 보증금 반환",

        # 권리/요건
        "전입신고 -> 주민등록",     
        "전입 -> 주민등록",
        "확정일자 -> 확정일자",    
        "대항요건 -> 대항요건",
        "대항력 -> 대항력",         
        "우선변제 -> 우선변제",
        "우선변제권 -> 우선변제권",

        # 절차/사건
        "조정위 -> 주택임대차분쟁조정위원회",  
        "분쟁조정 -> 주택임대차분쟁조정위원회",
        
        # [행위 - 돈]
        "올려 -> 증액", "인상 -> 증액", "더 달라고 -> 증액", "더 내라고 -> 증액",
        "깎아 -> 감액", "내려 -> 감액", "할인 -> 감액",
        "돌려주지 -> 반환", "안 줘 -> 반환거부", "떼먹 -> 미반환",
        
        # [행위 - 거주/계약]
        "계약서 -> 임대차계약증서", "특약 -> 특약사항",
        "이사 -> 주택의인도", "전입 -> 주민등록/대항력",
        "살고 -> 점유", "거주 -> 점유",
        "나가라 -> 계약갱신거절/명도", "나가라고 -> 계약갱신거절/명도", "비워 -> 명도", "쫓겨 -> 명도/대항력",
        "방 빼 -> 계약해지", "나갈게 -> 계약해지", "해지 -> 계약해지",
        "연장 -> 계약갱신", "더 살 -> 계약갱신", "재계약 -> 계약갱신",
        "수리 -> 수선의무/필요비", "고쳐 -> 수선의무", "물 새 -> 누수", "곰팡이 -> 하자",
        "청소비 -> 원상회복비용", "원상복구 -> 원상회복"
    ]

    prompt = ChatPromptTemplate.from_template(f'''사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요. 
    만약 변경할 필요가 없을 경우 사용자의 질문을 변경하지 마세요. 그런 경우에는 질문만 리턴해주세요
    사전 : {keyword_dict}
    질문 : {{question}}''')

    keyword_chain = prompt | llm | StrOutputParser()
    
    # 4. 질문을 표준화
    normalized_query = keyword_chain.invoke({'question':query}) # 출력함

    # 5. 관련문서 검색
    retriever = law_database.as_retriever(search_kwargs = {'k':k, 'filter':{'category':{'$in':categorize_content(query)}}})
    # 관련문서  rerank
    retrieved_docs = retriever.invoke(normalized_query)
    referenced_articles = extract_articles_from_docs(retrieved_docs) # 참조조항 출력

    # 프롬프트 템플릿 (위에것 그대로)을 이용한 rag체인 구성
    # template = f'당신은 최고의 한국 주택임대차보호법 전문가입니다. 다음 문맥을 참고하여 질문에 답하세요. 답을 모르면 모른다고 답하세요. 최대 3문장으로 간결하게 답변하세요. 질문 : {{query}} / 문맥 : {{context}} / 답변 :'
    template = f"""
    당신은 대한민국 부동산 법률(주택임대차보호법, 민법 등)에 정통한 'AI 법률 비서'입니다.
    사용자 질문에는 법률적 맥락을 돕기 위해 '(법률용어)'가 병기되어 있을 수 있습니다. 이를 참고하여 정확하게 분석하세요.

    [지시사항]
    1. **근거 기반**: 반드시 [참고 법령]에 기반하여 답변하세요.
    2. **입력 분석**: 질문에 포함된 괄호 안의 법률 용어(예: 명도, 대항력 등)를 핵심 키워드로 활용하세요.
    3. **답변 구조**:
       - 결론부터 명확히(가능/불가능) 제시
       - 📖 법적 근거(제N조) 상세 설명
       - 사용자 상황에 맞는 조언
    4. **🚨 위험 경고**: 불리한 특약이나 법 위반 사항(강행규정 위반)은 강력하게 경고하세요.
    5. **❗ 면책**: 법적 효력이 없음을 밝히세요.

    [참고 법령]
    {{context}}
    질문 : {{query}} / 답변 :
    """
    prompt = ChatPromptTemplate.from_template(template)
    prompt_chain = prompt | llm | StrOutputParser()
    result = prompt_chain.invoke({'context':format_documents(retrieved_docs),
                                  'query':query})
    
    print('원본 질문 :', query)
    print('표준화된 질문 :', normalized_query)
    print('='*90)
    print('\n✅ 답변 :', result)
    print('\n📌 참조사항 :', referenced_articles)
#     print('\n❗ 위 답변은 AI에 의해 생성된 답변이므로 약간의 오차가 날 수 있습니다 ❗')

In [26]:
# 사용예시

ask_with_reference('벽지 등이 손상될 경우 세입자가 전부 수리하고 나간다')

원본 질문 : 벽지 등이 손상될 경우 세입자가 전부 수리하고 나간다
표준화된 질문 : 벽지 등이 손상될 경우 임차인이 전부 수리하고 나간다

✅ 답변 : 결론: 불가능합니다. 

📖 법적 근거: 주택임대차보호법 제10조 제1항에서는 임대인은 장기임대차기간 동안 주택의 구조와 설비를 적절하게 유지할 의무가 있으며, 세입자는 임대인의 동의 없이 원상복구를 위한 수리를 종합적으로 진행할 수 없습니다. 즉, 주택의 손상이 발생한 경우 원칙적으로 임대인이 수선할 책임이 있습니다.

사용자 상황에 맞는 조언: 세입자가 벽지 등의 손상을 수리하고 나가겠다는 것은 법적으로 문제가 될 수 있습니다. 그러므로, 수리가 필요한 경우 임대인과 미리 협의하여 공통된 동의 아래 진행하는 것이 좋습니다. 만약 임대인이 수리를 하지 않으려 한다면, 세입자는 주택임대차보호법에 따라 임대인에게 수리 의무를 주장할 수 있습니다.

🚨 위험 경고: 만약 임대인이 요청하지 않은 상태에서 세입자가 수리를 강행한 경우, 해당 비용에 대한 반환을 요구하기 어려울 수 있으며, 임대인의 동의가 없을 시 불리한 계약 조건을 추후에 남길 수 있습니다. 따라서 주의가 필요합니다.

❗ 면책: 본 답변은 법적 효력을 갖지 않으며, 구체적인 법률 상담은 전문 변호사와 상담하시길 권장합니다.

📌 참조사항 : 주택임대차보호법


In [24]:
ask_with_reference('집주인이 월세를 올려 달래요. 제가 어렵다고 하니까 저보고 다음 달까지 나가라고 하네요. 계약서에는 그런 얘기가 없는데 이렇게 쫓겨나도 되나요?')

원본 질문 : 집주인이 월세를 올려 달래요. 제가 어렵다고 하니까 저보고 다음 달까지 나가라고 하네요. 계약서에는 그런 얘기가 없는데 이렇게 쫓겨나도 되나요?
표준화된 질문 : 임대인이 차임을 증액해 달래요. 제가 어렵다고 하니까 저보고 다음 달까지 나가라고 하네요. 임대차계약증서에는 그런 얘기가 없는데 이렇게 명도해도 되나요?

✅답변 : ### 결론
불가능합니다. 집주인이 임의로 퇴거를 요구할 수 없습니다.

### 법적 근거
주택임대차보호법 제6조 및 제11조에 따르면, 임차인은 임대차 계약의 기간 동안 안정적인 주거권을 보장받으며, 계약서를 위반하지 않는 한 임대인은 자의적으로 임차인을 쫓아낼 수 없습니다. 특히, 주택임대차보호법 제6조 제1항에서는 "임차인은 임대인의 동의 없는 한 임대차계약이 종료되지 않는다"는 내용을 담고 있습니다. 임대인은 임차인에게 다른 기간을 제시할 수 없고, 계약의 조건에 따라 요구할 수 없습니다.

### 상황에 맞는 조언
1. **계약서 검토**: 계약서에 월세 인상이나 퇴거에 대한 조항이 없으므로, 이를 근거로 집주인에게 반대의견을 제시하세요.
2. **주택임대차분쟁조정위원회 활용**: 만약 집주인이 퇴거를 강행하려 한다면, 주택임대차분쟁조정위원회에 중재를 요청하거나, 관련 법률 상담을 받는 것이 좋습니다.
3. **법적 대응**: 지속적으로 쫓겨날 위협이 있다면 필요시 전문 변호사의 상담을 통해 법적 조치를 고려해야 합니다.

### 🚨 위험 경고
퇴거를 요구하는 집주인의 주장이나 만약 따르지 않고 자발적으로 나가면, 불리한 증거가 남을 수 있으니 주의해야 합니다. 불리한 특약이나 위반 사항이 있을 경우, 법적 분쟁에 휘말릴 수 있으니 각별히 유의하시기 바랍니다.

### 면책
이 답변은 법적 자문이 아니며, 정확한 법률적 판단을 위해서는 전문 변호사와 상담하시기 바랍니다.

📌참조사항 : 주택임대차보호법

❗ 위 답변은 AI에 의해 생성된 답변이므로 약간의 오차가 날 수 있습니다 ❗
